# Importing and loading the libraries 

In [ ]:
!pip install Sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2MB 8.5MB/s 
     |████████████████████████████████| 2.5MB 8.8MB/s 
     |████████████████████████████████| 3.3MB 37.9MB/s 
     |████████████████████████████████| 901kB 43.1MB/s 


In [ ]:
!pip install -U spacy[cuda92]
!python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm
spacy.prefer_gpu()
spacy_nlp = en_core_web_sm.load()

     |████████████████████████████████| 6.4MB 6.8MB/s 
     |████████████████████████████████| 460kB 48.9MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 10.1MB 52.3MB/s 
     |████████████████████████████████| 624kB 42.2MB/s 
     |████████████████████████████████| 55.0MB 53kB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
2021-07-10 11:32:51.391604: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6M

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
import sentencepiece
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertConfig

## Checking the state of hardware accelerator

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## Mount the Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# BERT tranformer model traning and fine-tuning the hyper-parameters

## Setting the model name and additional model train parameter

In [ ]:
pretrainedModel = 'bert-base-cased'
batchSize = 16
seqLength = 512
DIR = os.path.expanduser('/content/gdrive/MyDrive/QG_dataset/')
tokenizer = BertTokenizer.from_pretrained(pretrainedModel)

## Defining the functions of evalSet class

In [ ]:
class evalSet(Dataset):
    def __init__(self, csv): 
        self.df = pd.read_csv(csv, engine = 'python')
        self.transforms = [self.shuffle, self.irrelevant]

    def __len__(self):
         return len(self.df)

    def __getitem__(self, idx): 
        _, question, answer = self.df.iloc[idx]
        label = random.choice([0, 1])

        if label == 0:
            question, answer = random.choice(self.transforms)(question, answer)
   
        encodedData = tokenizer(
            text = question,
            text_pair = answer,
            pad_to_max_length = True, 
            max_length = seqLength,
            truncation = True,
            return_tensors = "pt"
        )

        encodedData['input_ids'] = torch.squeeze(encodedData['input_ids'])
        encodedData['token_type_ids'] = torch.squeeze(encodedData['token_type_ids'])
        encodedData['attention_mask'] = torch.squeeze(encodedData['attention_mask'])
        return (encodedData.to(device), torch.tensor(label).to(device))
    
    def irrelevant(self, question, answer):
        doc = spacy_nlp(question)
        if len(doc.ents) > 1:
            replaceEntity = str(random.choice(doc.ents))
            for ent in doc.ents:
                question = question.replace(str(ent), replaceEntity)
        elif len(doc.ents) == 1:
            answer = str(doc.ents[0])
        return question, answer

trainSet = evalSet(os.path.join(DIR, 'qa_eval_train.csv')) 
trainLoader = DataLoader(trainSet, batchSize = batchSize, shuffle = True)
validSet = evalSet(os.path.join(DIR, 'qa_eval_valid.csv')) 
validLoader = DataLoader(validSet, batchSize = batchSize, shuffle = False)

## Training the model with bert-base pretrain model along with tokenizer

In [ ]:
epochs = 10
logInterval = 500
learningRate = 0.001

model = BertForSequenceClassification.from_pretrained(pretrainedModel)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learningRate)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

Saving the tokenizer files to the drive

In [ ]:
os.path.join(DIR, 'Tokenizer_Bert/')

'/content/gdrive/MyDrive/QG_dataset/Tokenizer_Bert/'

In [ ]:
# Saving the tokenizer files to the drive
tokenizer.save_pretrained(os.path.join(DIR, 'Tokenizer_Bert/'))

('/content/gdrive/MyDrive/QG_dataset/Tokenizer_Bert/tokenizer_config.json',
 '/content/gdrive/MyDrive/QG_dataset/Tokenizer_Bert/special_tokens_map.json',
 '/content/gdrive/MyDrive/QG_dataset/Tokenizer_Bert/vocab.txt',
 '/content/gdrive/MyDrive/QG_dataset/Tokenizer_Bert/added_tokens.json')

## Defining the training, evaluation funtion along with saving and loading the best model epoch

In [ ]:
modelName = 'qa_eval_model_trained_bert_large.pt'
modelPath = os.path.join(DIR, modelName)
modelPath

def train():
    model.train()
    totalLoss = 0.
    for batchIndex, batch in enumerate(trainLoader):
        data, labels = batch
        optimizer.zero_grad()
        output = model(**data, labels = labels)
        loss = output[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        totalLoss += loss.item()
        
        if batchIndex % logInterval == 0 and batchIndex > 0:
            curLoss = totalLoss / logInterval
            print('| Epoch {:3d} | {:2d}/{:2d} Batches | loss {:3.2f}'.format(epoch, batchIndex, len(trainLoader), curLoss))
            totalLoss = 0

In [ ]:
def evaluate(model, dataLoader):
    model.eval()
    totalScore = 0.
    with torch.no_grad():
        for batchIndex, batch in enumerate(dataLoader):
            data, labels = batch
            output = model(**data, labels=labels)
            preds = np.argmax(output[1].cpu(), axis=1)
            totalScore += (preds == labels.cpu()).sum()
    return totalScore / (len(dataLoader) * batchSize)

In [ ]:
def load():
    return torch.load(modelPath)

In [ ]:
def save(epoch, modelStateDict, optimizerStateDict, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': modelStateDict,
            'optimizer_state_dict': optimizerStateDict,
            'best_loss': loss,
            }, modelPath)

    print("| Model saved")

## Saving the best model based on accuracy and iterating over several epochs

In [ ]:
highestAccuracy = 0

accuracy = evaluate(model, validLoader)
print('| Before Training | Accuracy on Validation Set: {:3.7f}%'.format(accuracy))

for epoch in range(1, epochs + 1):
    train()
    accuracy = evaluate(model, validLoader)
    print('| End of Epoch {:4d} | Accuracy on Validation Set: {:3.7f}%'.format(epoch, accuracy))
    
    if accuracy > highestAccuracy:
        highestAccuracy = accuracy
        save(
             epoch, 
             model.state_dict(), 
             optimizer.state_dict(), 
             highestAccuracy
        )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


------------------------------------------------------------
| Before training | accuracy on valid set:  0.62%
------------------------------------------------------------
------------------------------------------------------------
| end of epoch   1 | accuracy on valid set:  0.67%
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
------------------------------------------------------------
| end of epoch   2 | accuracy on valid set:  0.71%
------------------------------------------------------------
| Model saved.
------------------------------------------------------------
------------------------------------------------------------
| end of epoch   3 | accuracy on valid set:  0.67%
------------------------------------------------------------
------------------------------------------------------------
| end of epoch   4 | accuracy on valid set:  0.67%
---------------------------------------------